In [ ]:
1) Diseñar una API a partir de un caso de uso que ustedes definen (por ejemplo, puede ser el caso de uso que han definido la semana pasada sobre el tema de SmartCity)

In [1]:
import mysql.connector
import pandas as pd
from IPython.display import display,HTML

conn = mysql.connector.connect(user="root",host="localhost",password="root")
cursor = conn.cursor()

cursor.execute("CREATE DATABASE IF NOT EXISTS RESTAURANT;")
cursor.execute("USE RESTAURANT;")
cursor.execute("SET FOREIGN_KEY_CHECKS=0;")
cursor.execute("DROP TABLE IF EXISTS ORDER_TABLE;")
cursor.execute("DROP TABLE IF EXISTS ITEM;")
cursor.execute("SET FOREIGN_KEY_CHECKS=1;")
cursor.execute("CREATE TABLE IF NOT EXISTS ORDER_TABLE(id INT NOT NULL AUTO_INCREMENT, \
               NOMBRE_CLIENTE VARCHAR(45), \
               PRIMARY KEY (id))ENGINE=InnoDB DEFAULT CHARSET=utf8;")
cursor.execute("CREATE TABLE IF NOT EXISTS ITEM(id INT NOT NULL AUTO_INCREMENT, \
               NOMBRE_ITEM VARCHAR(45) NOT NULL, \
               CANTIDAD INT NOT NULL, \
               FOREIGN KEY (id) REFERENCES ORDER_TABLE(id), \
               PRIMARY KEY (id, NOMBRE_ITEM))ENGINE=InnoDB DEFAULT CHARSET=utf8;")
cursor.execute("INSERT INTO ORDER_TABLE (id, NOMBRE_CLIENTE) VALUES (NULL, 'JUANITO');")
cursor.execute("INSERT INTO ORDER_TABLE (id, NOMBRE_CLIENTE) VALUES (NULL, 'PEDRITO');")
cursor.execute("INSERT INTO ORDER_TABLE (id, NOMBRE_CLIENTE) VALUES (NULL, 'JOSESITO');")
cursor.execute("INSERT INTO ORDER_TABLE (id, NOMBRE_CLIENTE) VALUES (NULL, 'PABLITO');")
cursor.execute("INSERT INTO ITEM(id, NOMBRE_ITEM, CANTIDAD) VALUES ((select id from ORDER_TABLE where NOMBRE_CLIENTE='JUANITO' LIMIT 1), 'LECHE', 1);")
cursor.execute("INSERT INTO ITEM(id, NOMBRE_ITEM, CANTIDAD) VALUES ((select id from ORDER_TABLE where NOMBRE_CLIENTE='PEDRITO' LIMIT 1), 'LECHE', 2);")
cursor.execute("INSERT INTO ITEM(id, NOMBRE_ITEM, CANTIDAD) VALUES ((select id from ORDER_TABLE where NOMBRE_CLIENTE='PEDRITO' LIMIT 1), 'CAFE', 4);")
cursor.execute("INSERT INTO ITEM(id, NOMBRE_ITEM, CANTIDAD) VALUES ((select id from ORDER_TABLE where NOMBRE_CLIENTE='JOSESITO' LIMIT 1), 'AZUCAR', 5);")
cursor.execute("INSERT INTO ITEM(id, NOMBRE_ITEM, CANTIDAD) VALUES ((select id from ORDER_TABLE where NOMBRE_CLIENTE='PABLITO' LIMIT 1), 'PAPAS', 2);")
cursor.execute("INSERT INTO ITEM(id, NOMBRE_ITEM, CANTIDAD) VALUES ((select id from ORDER_TABLE where NOMBRE_CLIENTE='PABLITO' LIMIT 1), 'PAN', 1);")


In [ ]:
2) Implementar su API

In [ ]:
import flask
import json
from flask import request, jsonify

app = flask.Flask(__name__)

@app.route('/', methods=['GET'])
def home():
    return '''
    <h1>RESTAURANT</h1>
    <h2>BIENVENIDO A LA API DEL RESTAURANT XD</h2>
    '''

@app.errorhandler(404)
def page_not_found(e):
    return "<h1>404</h1><p>The resource could not be found.</p>", 404

@app.route('/api/items')
def items():
    query = "SELECT * FROM ITEM"
    query_parameters = request.args
    nombre = query_parameters.get('name')
    cantidad = query_parameters.get('cantidad')
    if (nombre or cantidad):
        query += " WHERE "
    if (nombre):
        query += "NOMBRE_ITEM='" + nombre +"'"
    if (cantidad):
        if (nombre):
            query += " AND"
        query += " CANTIDAD=" + cantidad
    query += ";"
    print(query)
    cursor.execute(query)
    its = cursor.fetchall()
    row_headers=[x[0] for x in cursor.description]
    json_data=[]
    for result in its:
        json_data.append(dict(zip(row_headers,result)))
    return jsonify(json_data)
@app.route('/api/orders')
def ordersAll():
    query = "SELECT * FROM ITEM INNER JOIN ORDER_TABLE ON ITEM.id = ORDER_TABLE.id;"
    print(query)
    cursor.execute(query)
    its = cursor.fetchall()
    row_headers=[x[0] for x in cursor.description]
    json_data=[]
    for result in its:
        json_data.append(dict(zip(row_headers,result)))
    return jsonify(json_data)
@app.route('/api/orders/<name>')
def ordersSpecific(name):
    query = "SELECT * FROM ITEM INNER JOIN ORDER_TABLE ON ITEM.id = ORDER_TABLE.id WHERE NOMBRE_CLIENTE='"
    query += name + "';"
    print(query)
    cursor.execute(query)
    its = cursor.fetchall()
    row_headers=[x[0] for x in cursor.description]
    json_data=[]
    for result in its:
        json_data.append(dict(zip(row_headers,result)))
    return jsonify(json_data)
app.run(debug=False,port=1234)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:1234/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Nov/2018 16:53:32] "GET /api/items HTTP/1.1" 200 -


SELECT * FROM ITEM;


127.0.0.1 - - [04/Nov/2018 16:53:38] "GET /api/orders HTTP/1.1" 200 -


SELECT * FROM ITEM INNER JOIN ORDER_TABLE ON ITEM.id = ORDER_TABLE.id;


127.0.0.1 - - [04/Nov/2018 16:54:07] "GET /api/orders/JUANITO HTTP/1.1" 200 -


SELECT * FROM ITEM INNER JOIN ORDER_TABLE ON ITEM.id = ORDER_TABLE.id WHERE NOMBRE_CLIENTE='JUANITO';


In [ ]:
3) ¿Cómo podrían desplegar su API más allá de su computador local?

In [ ]:
Podriamos desplegar la API de dos formas principalmente, primero, 
puede ser designando un computador como servidor habiendo previamente 
abierto los puertos del router al internet, y dejando el servidor 
funcionando de forma abierta en el puerto abierto.
La segunda forma es consiguiendo un servidor remoto como por ejemplo AWS
el cual está diseñado para esta funcion, el cual se le sube el script
de la API y se deja disponible al internet.


In [ ]:
4) ¿Observan algunas limitaciones al momento de crear una API web basada en MySQL?

In [ ]:
La principal limitación que puedo observas es del hecho que la información no se maneja
con el formato json, lo que es contraproducente con como se maneja la informacion
generalmente en los protocolos web normales, lo que significa que antes de devolver
la informacion de la solicitud siempre se requiere de un paso extra de parseo manual
o semi-automatico a la forma que se desea manejar.
La segunda limitación es que para hacer cada endpoint significa un tipo de solicitud
distinto a la base de datos, lo cual implica un trabajo extra pensando en los posibles
joins y subconsultas que la solicitud necesite.

In [ ]:
5) ¿Qué es el movimiento de <i>Open Data</i>? ¿En qué sentido podría aportar a la ciudad de Valdivia?

In [ ]:
El movimiento Open Data tengo entendido que hace referencia al movimiento de dejar "disponible" la informacion
de las bases de datos al publico en general, sin mayores obstaculos, lo cual contribuye a la posibilidad
de mayores y mejores iniciativas a la innovación.